In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import autobot

import torch
from torch import nn
import numpy as np

In [3]:
batch_size = 4
num_view = 3
h, w = 256, 256

imgs = torch.rand((batch_size, num_view, 3, h, w))

In [4]:
emb_layer = autobot.MultiViewEmbedding(emb_dim=128, patch_size=(16, 16))
emb = emb_layer(imgs)
print(emb.shape)

torch.Size([4, 3, 256, 128])


In [44]:
class MultiViewAttention(nn.Module):
    '''
        MVA searches object in 3D space by computing similarity 
    '''
    def __init__(self,
        emb_dim, attn_dim, v_dim, num_heads,
        num_view, view_transformation=None
    ):
        super().__init__()

        self.emb_dim = emb_dim
        self.attn_dim = attn_dim
        self.v_dim = v_dim
        self.num_heads = num_heads

        self.query = nn.Linear(emb_dim, attn_dim)
        self.key = nn.Linear(emb_dim, attn_dim)
        self.value = nn.Linear(emb_dim, v_dim)
        self.linear = nn.Linear(v_dim*num_heads, v_dim*num_heads)

        self.per_view_mha = nn.ModuleList([
            autobot.MultiHeadAttention(emb_dim, attn_dim, v_dim, num_heads)
            for _ in range(num_view)
        ])
        
    def forward(self, v, vs):
        query = self.query(v)
        
        print('query:', v.shape, query.shape)
        print('-'*30)
        key = self.key(vs)
        value = self.value(vs)
        print('key, value:', vs.shape, key.shape, value.shape)

        print(query.unsqueeze(1).shape)
        logits = torch.matmul(query.unsqueeze(1), key.transpose(-1, -2))
        print(logits.shape)
        logits_ = logits.sum(1)
        print(logits.shape)
        attn_ = logits_.softmax(dim=-1)

        attn = logits.softmax(dim=-1)
        v = torch.matmul(attn, value)
        print('v', v.shape)

        o = torch.matmul(attn_, v.sum(1))
        print('o', o.shape)

        return self.linear(o)


mva_layer = MultiViewAttention(emb_dim=128, attn_dim=8, v_dim=16, num_heads=1, num_view=num_view)
i = 0
print(emb.shape)
mva = mva_layer(
    emb[:, i, :, :], 
    torch.cat([
        emb[:, :i, :, :],
        emb[:, i+1:, :, :]
    ], dim=1)  # from i-th view to other views that are not i-th
)
print(mva.shape)

torch.Size([4, 3, 256, 128])
query: torch.Size([4, 256, 128]) torch.Size([4, 256, 8])
------------------------------
key, value: torch.Size([4, 2, 256, 128]) torch.Size([4, 2, 256, 8]) torch.Size([4, 2, 256, 16])
torch.Size([4, 1, 256, 8])
torch.Size([4, 2, 256, 256])
torch.Size([4, 2, 256, 256])
v torch.Size([4, 2, 256, 16])
o torch.Size([4, 256, 16])
torch.Size([4, 256, 16])


In [2]:
K = [[ 282.363047,      0.,          166.21515189],
     [   0.,          280.10715905,  108.05494375],
     [   0.,            0.,            1.        ]]
K = np.array(K)
R = np.eye(3)
t = np.array([[0],[1.],[0]])
P = K.dot(np.hstack((R,t)))
C = np.array([0., 0., 1.])
p1 = np.array([215, 180, 1.])

In [3]:
P

array([[282.363047  ,   0.        , 166.21515189,   0.        ],
       [  0.        , 280.10715905, 108.05494375, 280.10715905],
       [  0.        ,   0.        ,   1.        ,   0.        ]])

In [4]:
np.hstack((R,t))

array([[1., 0., 0., 0.],
       [0., 1., 0., 1.],
       [0., 0., 1., 0.]])